In [1]:
import os
!export CUDA_VISIBLE_DEVICES=2
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
%load_ext autoreload
%autoreload 2

import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List
from utils import *

import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
import torchvision.models as models

from torchprofile import profile_macs
# from torch.nn.parallel import DistributedDataParallel as DDP
# import torch.distributed as dist
from torch.nn.parallel import DataParallel


assert torch.cuda.is_available(), \
"CUDA support is not available."

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [4]:
dataloader = get_dataloader("imagenet", batch_size=256)

In [5]:
base_model = models.vgg16(pretrained=True).cuda()

/home/shariff/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shariff/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
evaluate_model_imagenet(base_model, dataloader, count_nonzero_only=True)

eval:   0%|                                                                                                                                                                             | 0/501 [00:00<?, ?it/s]

model has test accuracy=68.74%


model has train accuracy=77.84%
model has size=527.79 MiB
model has macs=15.47 Gmacs
average inference time is 0.0023 seconds
model has 138.36 M parameters


In [7]:
print(base_model)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
# for param in base_model.features.parameters():
#     param.requires_grad = False

In [9]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
# # Move the model to the GPU if available
# base_model.to(device)

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(base_model.parameters(), lr=0.001, momentum=0.9)

# # Training loop
# num_epochs = 10



In [11]:
# for epoch in range(num_epochs):
#     base_model.train()  # Set the model to training mode
#     running_loss = 0.0
    
#     for images, labels in dataloader['train']:
#         images, labels = images.to(device), labels.to(device)
        
#         optimizer.zero_grad()
        
#         outputs = base_model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
    
#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(dataloader['train'])}")

# print("Training complete!")


In [12]:
# You can save the fine-tuned model
# torch.save(base_model.state_dict(), 'fine_tuned_vgg16.pth')

In [13]:
# evaluate_model_imagenet(base_model, dataloader, count_nonzero_only=True)

In [14]:
# dataloader = get_dataloader("imagenet", batch_size=1)

In [15]:
# train_dataloader = dataloader['train']

In [16]:
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img.reshape((224,224,3)), cmap="gray")
# plt.show()
# print(f"Label: {label}")

In [17]:
# from tqdm import tqdm

In [18]:
# classes = []
# for data in tqdm(train_dataloader):
#     classes.append(int(data[1][0]))


In [19]:
# len(list(set(classes)))

In [20]:
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()


In [21]:
# img.permute(1,2,0).shape

In [22]:
# len(train_dataloader)/256

In [27]:
compressed_model = copy.deepcopy(base_model)
compressed_model.classifier._modules[str(4)] = nn.LeakyReLU(negative_slope=nn.Parameter(torch.tensor(1), requires_grad=False), inplace=True)
compressed_model.classifier._modules[str(1)] = nn.LeakyReLU(negative_slope=nn.Parameter(torch.tensor(1), requires_grad=False), inplace=True)
compressed_model = collapse_layers_imagenet_vgg16(compressed_model, 4)


# compressed_model.load_state_dict(torch.load('temp_best_model.pth'))
w0 = compressed_model.classifier._modules[str(0)].weight.data
w6 = compressed_model.classifier._modules[str(6)].weight.data
b0 = compressed_model.classifier._modules[str(0)].bias.data
b6 = compressed_model.classifier._modules[str(6)].bias.data
# model.classifier._modules[str(relu_id + 2)].weight.data = torch.matmul(w3, w0)
compressed_model.classifier._modules[str(6)].weight.data = torch.matmul(w6, w0)
# model.classifier._modules[str(relu_id + 2)].bias.data = torch.matmul(w3, b0) + b3
compressed_model.classifier._modules[str(6)].bias.data = torch.matmul(w6, b0) + b6

# model.classifier._modules[str(relu_id - 1)] = nn.Identity()
# model.classifier._modules[str(relu_id)] = nn.Identity()
# model.classifier._modules[str(relu_id + 1)] = nn.Identity()
compressed_model.classifier._modules[str(0)] = nn.Identity()
compressed_model.classifier._modules[str(1)] = nn.Identity()
compressed_model.classifier._modules[str(2)] = nn.Identity()

In [28]:
evaluate_model(compressed_model, dataloader, count_nonzero_only=True)


model has test accuracy=33.06%
model has train accuracy=36.21%
model has size=151.84 MiB
model has macs=0.34 Gmacs
average inference time is 0.0013 seconds
model has 39.80 M parameters


In [29]:
print(f"model has {get_num_parameters(compressed_model, True)/1e6:.2f} M parameters")

model has 39.80 M parameters


In [30]:
compressed_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [31]:
torch.save(compressed_model.state_dict(), "student_vgg16.pt")

In [49]:
start = time.time()

In [50]:
end = time.time()

In [51]:
end-start

0.08238410949707031